# Regresores. Estudio de outliers

En este notebook se realizará un estudio de los outliers del conjunto completo de datos.

In [1]:
import pandas as pd
import numpy as np
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.linear_model import SGDRegressor
from sklearn.svm import SVR
from explainerdashboard import RegressionExplainer, ClassifierExplainer, ExplainerDashboard
from math import sqrt

RANDOM_STATE = 123

Cargamos los datos y eliminamos las columnas con gran cantidad de nulos, fechas, descripción en lenguaje natural y repetidas en ambos datasets. También eliminamos las columnas con gran cantidad de valores, ya que analizando los resultados del notebook anterior, decidí continuar sin esas columnas

In [2]:
smt_2020_df = pd.read_excel('data/I11_SMT Incidents 2020.xlsx')

smt_2020_df.columns = smt_2020_df.iloc[11].values
smt_2020_df = smt_2020_df.iloc[12:]
smt_2020_df = smt_2020_df.reset_index(drop=True)


c:\ProgramData\Anaconda3\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning: Workbook contains no default style, apply openpyxl's default


In [3]:
smt_2021_df = pd.read_excel('data/I11_SMT Incidents 2021.xlsx')

smt_2021_df.columns = smt_2021_df.iloc[11].values
smt_2021_df = smt_2021_df.iloc[12:]
smt_2021_df = smt_2021_df.reset_index(drop=True)

In [4]:
smt_2022_df = pd.read_excel('data/I11_SMT Incidents 2022.xlsx')

smt_2022_df.columns = smt_2022_df.iloc[11].values
smt_2022_df = smt_2022_df.iloc[12:]
smt_2022_df = smt_2022_df.reset_index(drop=True)

In [5]:
smt_df = pd.concat([smt_2020_df, smt_2021_df, smt_2022_df])
smt_df.reset_index(drop=True,inplace=True)

pd.set_option('display.max_columns', None)

smt_df

,Incident ID,Original Incident Number,Requisition ID,Created Date (UTC+0),Open Incident Type,Current Incident Type,Status,Status Reason,Company,Customer Department,Region,Site Group,Site,Desk Location,Reported Source,Summary,Impact,Open Priority,Current Priority,Assigned Group,Assigned Group Department,In Progess Time (hhh:mi)\n,Actual Duration/Open Time (hhh:mi),Down Time of CI-Unavailability (hhh:mi),Last Modified Date (UTC+0),First Resolved Date (UTC+0),Last Resolved Date (UTC+0),Created Month,Operational Categorization Tier 1,Operational Categorization Tier 2,Operational Categorization Tier 3,Service+,CI+,Product Categorization Tier 1,Product Categorization Tier 2,Product Categorization Tier 3,Product Name,Resolution Categorization Tier 1,Resolution Categorization Tier 2,Resolution Categorization Tier 3,Resolution Product Categorization Tier 1,Resolution Product Categorization Tier 2,Resolution Product Categorization Tier 3,Resolution Product Name,Primary Center Code,Target Date,Notes,Resolution,Created by CI-Hotline
0,INC000017348507,NaN,NaN,02.01.2020 11:00,Not assigned,User Service Request,Closed,No Further Action Required,AE,RBEM/MSS4-Q,EMEA,ES,MD1,MD1 150P3,Phone,Ampliar fecha de validez de windows#,4-Minor/Localized,Medium,Medium,Iberia4 -Md-,CI/CER-IB,Not Defined,0260:05,Not Specified,31.03.2020 22:04,03.01.2020 07:21,13.01.2020 07:05,JAN 2020,Request,- None -,- None -,CI-CAT ADMINISTRATION |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,User Rights,- None -,Service,Infrastructure Service,Support Service,NaN,965899,,I: Important: I: Do NOT use this service for r...,"Se habilita la cuenta NT, esta se desactivo ce...",Yes
1,INC000017354421,NaN,NaN,04.01.2020 17:52,Not assigned,User Service Request,Closed,No Further Action Required,AE,RBEM/MSS4-Q,EMEA,ES,MD1,Md1 150P3,Phone,Cuenta de NT caducada#,4-Minor/Localized,Low,Medium,Iberia4 -Md-,CI/CER-IB,Not Defined,0205:20,Not Specified,31.03.2020 22:04,05.01.2020 11:48,13.01.2020 07:12,JAN 2020,Request,Modify,- None -,NT USER MODIFY |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,User Rights,- None -,Service,Infrastructure Service,Support Service,NaN,965859,,30.06.2020 Q: User ID (e.g. xyz8fe): A: GSS2M...,Se amplía de nuevo la fecha de validez del usu...,Yes
2,INC000017400389,NaN,NaN,13.01.2020 09:51,Not assigned,User Service Request,Closed,Customer Follow-Up Required,AE,RBEM/LOP1,EMEA,ES,MD1,Md1 155P3,Email,usuario caducado#,4-Minor/Localized,Low,Medium,Iberia4 -Md-,CI/CER-IB,0:4,0022:02,Not Specified,31.03.2020 22:04,13.01.2020 10:39,14.01.2020 07:53,JAN 2020,Request,- None -,- None -,CI-CAT ADMINISTRATION |IBERIA4,NaN,Service,Infrastructure Service,Support Service,NaN,Request,- None -,- None -,Service,Infrastructure Service,Support Service,NaN,965138,,Su solicitud vía e-mail ha sido transferida a ...,"Como ya dijimos al cerrar el ticket, la valide...",Yes
3,INC000017470045,NaN,10065659,23.01.2020 14:49,Not assigned,User Service Request,Closed,No Further Action Required,M,M/PQA1-RBEM,EMEA,ES,MD1,Md1 162P4,Walk In,My Mobile MSD Micro Pilot : Order Service#,4-Minor/Localized,Low,Low,Iberia4 -Md-,CI/CER-IB,170:8,0502:49,Not Specified,19.03.2020 22:00,13.02.2020 13:38,13.02.2020 13:38,JAN 2020,Request,- None -,- None -,MY MOBILE - LOCAL SUPPORT |IBERIA4,NaN,Service,End Device Service,Mobile Device Service,NaN,Request,- None -,- None -,Service,End Device Service,Mobile Device Service,NaN,NaN,,------------------------------------------ // ...,Móvil MD-Y1733 entregado y configurado,Yes
4,INC000017470799,NaN,10061208,23.01.2020 16:51,Not assigned,User Service Request,Closed,No Further Action Required,TT,TT/SSP-ASA,EMEA,ES,MD,Md,Other,My Mobile MSD Micro Pilot : Order Service#,4-Minor/Localized,Low,Low,Iberia4 -Md-,CI/CER-IB,360:45,0475:00,Not Specified,18.03.2020 22:00,12.02.2020 11:51,12.02.2020 11:51,JAN 2020,Request,- None -,- None -,MY MOBILE - LOCAL SUPPORT |IBERIA4,NaN,Service,End Device Service,Mobile Device Service,NaN,Requ

In [6]:
celonis_2020_df = pd.read_excel('data/Tickets_Celonis_2020.xlsx')
celonis_2020_df = celonis_2020_df.reset_index(drop=True)

In [7]:
celonis_2021_df = pd.read_excel('data/Tickets_Celonis_2021.xlsx')
celonis_2021_df = celonis_2021_df.reset_index(drop=True)

In [8]:
celonis_2022_df = pd.read_excel('data/Tickets_Celonis_2022.xlsx')
celonis_2022_df = celonis_2022_df.reset_index(drop=True)

In [9]:
celonis_df = pd.concat([celonis_2020_df, celonis_2021_df, celonis_2022_df])
celonis_df.reset_index(drop=True,inplace=True)

celonis_df

,Incident Number,Submit Date,Service Name,Site Group,Site,Incident Type,Ticket Summary,Submitter Group,Last Assigned Organization,Last Assigned Group,Entry Channel,Last Resolution Date,Template Name,User Company,Resolver Organization,Requisition ID,Reported Source,Last Resolution Time,CI,Status,Priority
0,INC000017346024,2020-01-02 05:46:38,CI-CAT ADMINISTRATION |IBERIA4,ES,Md1,User Service Request,Usuarios expirados,ITSD - IT Service Desk,Regional Services World,Iberia4 -Md-,CI-Hotline / ITSD,2020-01-02 10:36:24,NaN,AE,IT Personal Support,NaN,Phone,4.956944,NaN,Closed,Medium
1,INC000017346091,2020-01-02 06:42:34,CI-CAT ADMINISTRATION |IBERIA4,ES,Md,User Service Request,Password NT - Reset_ES,ITSD - IT Service Desk,Regional Services World,Iberia4 -Md-,CI-Hotline / ITSD,2020-01-02 07:11:50,Password NT - Reset_ES,RBIB,IT Personal Support,NaN,Phone,0.514444,NaN,Closed,Medium
2,INC000017346702,2020-01-02 06:57:03,CI-CAT ADMINISTRATION |IBERIA4,ES,Md1,User Service Request,Ampliación de validez de cuenta,ITSD - IT Service Desk,Regional Services World,Iberia4 -Md-,CI-Hotline / ITSD,2020-01-02 07:08:31,NaN,AE,IT Personal Support,NaN,Phone,0.260278,NaN,Closed,Medium
3,INC000017346738,2020-01-02 07:31:46,CI-CAT ADMINISTRATION |IBERIA4,ES,Md1,User Service Request,Cuenta windows expirada,ITSD - IT Service Desk,Regional Services World,Iberia4 -Md-,CI-Hotline / ITSD,2020-01-03 11:00:32,NaN,AE,IT Personal Support,NaN,Phone,27.522500,NaN,Closed,Low
4,INC000017346753,2020-01-02 07:55:23,CI-CAT ADMINISTRATION |IBERIA4,ES,Md,User Service Request,Password NT - Reset_ES,ITSD - IT Service Desk,Regional Services World,Iberia4 -Md-,CI-Hotline / ITSD,2020-01-02 08:46:09,Password NT - Reset_ES,AA,IT Personal Support,NaN,Phone,0.860000,NaN,Closed,Medium
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14733,INC000023672067,2022-12-27 15:59:26,CI-CAT ADMINISTRATION |SPAIN1,ES,Aj,User Service Request,Mailbox - ITSP Request 19405191,ITSD - IT Service Desk,Regional Services EMEA,Spain1,IT Service Portal,2023-01-03 12:12:35,NaN,PS,IT Personal Support,19405191.0,Self Service,164.219167,NaN,Closed,Low
14734,INC000023672229,2022-12-27 15:06:34,SOFTWARE NO SCCM |SPAIN1,ES,Md,User Service Restoration,wmedtra not working,ITSD - ES,Regional Services EMEA,Spain1,CI-Hotline / ITSD,2022-12-28 14:58:20,NaN,C,IT Personal Support,NaN,Phone,23.933889,SOFTWARE NO SCCM |SPAIN1,Closed,Low
14735,INC000023674299,2022-12-28 12:26:54,TELEPHONY LOCAL SUPPORT |SPAIN1,ES,Md,User Service Restoration,Mobile phone,ITSD - IT Service Desk,Regional Services EMEA,Spain1,CI-Hotline / ITSD,2022-12-28 12:57:20,NaN,GS,IT Personal Support,NaN,Phone,0.578333,TELEPHONY LOCAL SUPPORT |SPAIN1,Closed,Low
14736,INC000023674336,2022-12-28 09:18:41,"NT PASSWORD RESET |EMEA, LA",ES,Md1,User Service Request,Reset password,Spain1,Regional Services EMEA,Spain1,IT Personal Support,2022-12-28 09:18:41,CD/CW_only_ITS_Request,AE,IT Personal Support,NaN,Walk In,0.020556,"NT PASSWORD RESET |EMEA, LA",Closed,Low


In [10]:
celonis_df = celonis_df.drop(['Template Name', 'Requisition ID', 'CI'], axis=1)
celonis_df.dropna(inplace=True)
celonis_df = celonis_df.drop(['Submit Date', 'Last Resolution Date', 'Ticket Summary'], axis=1)

In [11]:
smt_df = smt_df.drop(['Original Incident Number', 'Requisition ID', 'CI+', 'Product Name', 
                      'Resolution Product Name', 'Primary Center Code'], axis=1)
smt_df.dropna(inplace=True)
smt_df = smt_df.drop(['Created Date (UTC+0)', 'Summary', 'Last Modified Date (UTC+0)', 
                      'First Resolved Date (UTC+0)', 'Last Resolved Date (UTC+0)', 'Notes', 'Resolution'], axis=1)

In [12]:
smt_df = smt_df.drop(['Site Group', 'Site', 'Reported Source', 'Status'], axis=1)

In [13]:
combined_df = pd.merge(celonis_df, smt_df, how='inner', left_on='Incident Number', right_on='Incident ID')
combined_df.drop(['Service Name', 'Incident ID','In Progess Time (hhh:mi)\n', 'Actual Duration/Open Time (hhh:mi)', 'Customer Department', 'Desk Location', 
                  'Down Time of CI-Unavailability (hhh:mi)', 'Service+', 'Target Date'], axis=1, inplace=True)

In [14]:
combined_df.shape

(11687, 38)

In [15]:
columns_list = combined_df.columns
columns_list = [elem for elem in columns_list if elem not in ['Incident Number', 'Last Resolution Time']] 
combined_df_one_hot = pd.get_dummies(combined_df, columns=columns_list)
combined_df_one_hot.shape

(11687, 324)

Dividimos el conjunto de datos en entrenamiento y prueba.

In [16]:
X = combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).values
y = combined_df_one_hot['Last Resolution Time'].values

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=RANDOM_STATE)

Obtenemos los 10 mayores valores de Last Resolution Time.

In [17]:
top10_df = pd.DataFrame({'Last Resolution Time': y_train}).nlargest(10, 'Last Resolution Time').reset_index(drop=True)
top10_df.index += 1
top10_df

,Last Resolution Time
1,4488.650000
2,4488.626111
3,3950.865278
4,3549.974167
5,3342.063889
6,2740.438889
7,2543.210278
8,2328.255833
9,2328.212778
10,2034.331667


Eliminamos el 2% superior de los outliers para Last Resolution Time.

In [18]:
umbral_top_percent = np.percentile(y_train, 98)
to_delete = np.where(y_train >= umbral_top_percent)
X_train = np.delete(X_train, to_delete, axis=0)
y_train = np.delete(y_train, to_delete)

Volvemos a obtener los 10 mayores valores de Last Resolution Time.

In [19]:
top10_df2 = pd.DataFrame({'Last Resolution Time': y_train}).nlargest(10, 'Last Resolution Time').reset_index(drop=True)
top10_df2.index += 1
top10_df2

,Last Resolution Time
1,363.895556
2,363.525278
3,362.711944
4,361.865556
5,360.643611
6,360.131667
7,359.998611
8,359.327222
9,358.983333
10,358.760278


Observamos como se han reducido considerablemente al eliminar los outliers. Obtenemos una tabla para comparar los 10 mayores valores de Last Resolution Time antes y después de eliminar outliers.

In [20]:
to10_comb_df = pd.concat([top10_df, top10_df2], axis=1)
to10_comb_df.columns = ['Con Outliers', 'Sin Outliers']
multi_index_cols = pd.MultiIndex.from_product([['Last Resolution Time'], ['Con Outliers', 'Sin Outliers']])
to10_comb_df.columns = multi_index_cols
to10_comb_df

Last Resolution Time             
           Con Outliers Sin Outliers
1           4488.650000   363.895556
2           4488.626111   363.525278
3           3950.865278   362.711944
4           3549.974167   361.865556
5           3342.063889   360.643611
6           2740.438889   360.131667
7           2543.210278   359.998611
8           2328.255833   359.327222
9           2328.212778   358.983333
10          2034.331667   358.760278

Creación, entrenamiento y evaluación del modelo de GradientBoosting

In [21]:
gbr = GradientBoostingRegressor(random_state=RANDOM_STATE)

gbr.fit(X_train, y_train)

predictions = gbr.predict(X_test)

mse = mean_squared_error(y_test, predictions)
mae = mean_absolute_error(y_test, predictions)
print("Error cuadrático medio (MSE) en el conjunto de prueba: {:.4f}".format(mse))
print("Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: {:.4f}".format(sqrt(mse)))
print("Error medio absoluto (MAE) en el conjunto de prueba: {:.4f}".format(mae))

Error cuadrático medio (MSE) en el conjunto de prueba: 14143.3241
Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: 118.9257
Error medio absoluto (MAE) en el conjunto de prueba: 35.8753


Creación, entrenamiento y evaluación del modelo de Stochastic Gradient Descent

In [22]:
sgd = SGDRegressor(random_state=RANDOM_STATE)

sgd.fit(X_train, y_train)

sgd_predictions = sgd.predict(X_test)

mse = mean_squared_error(y_test, sgd_predictions)
mae = mean_absolute_error(y_test, sgd_predictions)
print("Error cuadrático medio (MSE) en el conjunto de prueba: {:.4f}".format(mse))
print("Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: {:.4f}".format(sqrt(mse)))
print("Error medio absoluto (MAE) en el conjunto de prueba: {:.4f}".format(mae))

Error cuadrático medio (MSE) en el conjunto de prueba: 14213.8252
Raíz del error cuadrático medio (RMSE) en el conjunto de prueba: 119.2217
Error medio absoluto (MAE) en el conjunto de prueba: 40.3002


Creamos los dataframes para los explainers

In [108]:
X_test_df = pd.DataFrame(X_test, columns=combined_df_one_hot.drop(['Incident Number', 'Last Resolution Time'], axis=1).columns)
y_test_df = pd.DataFrame(y_test, columns=['Last Resolution Time'])

Creamos los cuadros de mando de explainer dashboard para analizar los modelos y obtener conclusiones

In [26]:
explainer = RegressionExplainer(gbr, X_test_df, y_test_df)

ExplainerDashboard(explainer).run(port = 8057)

Generating self.shap_explainer = shap.TreeExplainer(model)
Building ExplainerDashboard..
Detected notebook environment, consider setting mode='external', mode='inline' or mode='jupyterlab' to keep the notebook interactive while the dashboard is running...
The explainer object has no decision_trees property. so setting decision_trees=False...
Generating layout...
Calculating shap values...


Calculating predictions...
Calculating residuals...
Calculating absolute residuals...
Calculating shap interaction values...
Reminder: TreeShap computational complexity is O(TLD^2), where T is the number of trees, L is the maximum number of leaves in any tree and D the maximal depth of any tree. So reducing these will speed up the calculation.


In [ ]:
explainer = RegressionExplainer(sgd, X_test_df, y_test_df)

ExplainerDashboard(explainer).run(port = 8058)